<a href="https://colab.research.google.com/github/mahirbarot/gujarati_pos/blob/main/gujarati_pos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re
import numpy as np
import pandas as pd

In [ ]:
filename='/content/2_Gujarati Tourism POS Tagged Corpus.txt'
with open(filename, 'r', encoding='utf-8') as f:
    corpus = f.read()

# new_data=re.sub(r'[^A-Z\\\u0A80-\u0AFF_ ]','',corpus)
new_data=re.sub(r'[a-z(0-9)?]','',corpus)
# new_data=re.sub(r'["IDVa-z0-9"]','',corpus)
#[\u0900-\u097F]
# new_data=re.sub(r'[^_A-Z\\ ]','',corpus)
#  = re.sub(r'[^]+', ' ', text)

print(new_data)

with open('data.txt','w') as f:
  f.write(new_data)
  f.close()



In [ ]:
filename='data.txt'
# with open(filename, 'r', encoding='utf-8-sig') as f:
with open(filename, 'r', encoding='utf-8-sig') as f:
    clean_corpus = f.read().strip()
    

key_val=clean_corpus.split(" ")

In [ ]:
x=[]
y=[]
for i in range(len(key_val)):
  vals=key_val[i].split('\\')
  if len(vals)==2:
    vals[0]=re.sub('\s+', '', vals[0])
    vals[1]=re.sub('\s+', '', vals[1])
    x.append(vals[0])
    y.append(vals[1])

print(x[-5:])
print(y[-5:])

['સુરક્ષિત', 'રાખવામાં', 'આવી', 'છે', '']
['JJ', 'V_VM', 'V_VAUX', 'V_VAUX', 'RD_PUNC']


In [ ]:
# df = pd.DataFrame(x,y)
# df.to_csv('my_data.csv', index=False)
print(x[0:5])

['સપ્તપુરીઓના', 'દર્શનથી', 'મળે', 'છે', 'તીર્થનું']


In [ ]:
#code for converting lists to column
percentile_list = pd.DataFrame(np.column_stack([x,y]), 
                               columns=['x','y'])
print(x[0:5])

['સપ્તપુરીઓના', 'દર્શનથી', 'મળે', 'છે', 'તીર્થનું']


In [ ]:
df=pd.read_csv('my_data.csv')
df.head()
percentile_list.tail()
percentile_list.to_csv('clean_data_gujPOS.csv', index=False)

In [ ]:
df=pd.read_csv('/content/clean_data_gujPOS.csv')
x=df['x'].astype('str')
y=df['y']

In [ ]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, Dropout
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Define the input data

# Convert the labels to numeric categories
y = np.squeeze(y)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

# Tokenize the input data

tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
x = tokenizer.texts_to_sequences(x)


# Pad the sequences to a fixed length
max_len = 50  # maximum length of a sentence
x = pad_sequences(x, maxlen=max_len, padding='post')

# Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Define the neural network model
model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=64, input_length=max_len))
model.add(LSTM(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(Dense(units=32, activation='relu'))
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(x_train, np.array(y_train), epochs=10, batch_size=32, validation_split=0.1)

# Evaluate the model
test_loss, test_acc = model.evaluate(x_test, np.array(y_test))
print('Test accuracy:', test_acc)

# Use the model to predict POS tags for new input text
new_text = [...]  # list of new Gujarati sentences
new_x = tokenizer.texts_to_sequences(new_text)
new_x = pad_sequences(new_x, maxlen=max_len, padding='post')
pred_y = model.predict(new_x)
pred_y = [label_encoder.inverse_transform(np.argmax(pred, axis=-1)) for pred in pred_y]
print('Predicted POS tags:', pred_y)

AttributeError: ignored